* Instructor is open source library help us to generate structure output from "model providers" that don't support formal structured outputs.

* Follows retry based/reprompting based method, if model output doesn't match the required format.

In [143]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Loading Model API Key:

In [144]:
import requests
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv()
groq_api_key=os.environ['GROQ_API_KEY']
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in environment variables")

url = "https://api.groq.com/openai/v1/models"
headers = {
    "Authorization": f"Bearer {groq_api_key}",
    "Content-Type": "application/json"
}

In [145]:
from groq import Groq
client = Groq()
chat_completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[{
        "role": "user", 
        "content": 'sup'
    }]
)

chat_completion.choices[0].message.content

"Sup back at ya! What's on your mind?"

### Adding Instructor:

In [146]:
import instructor
from instructor import from_openai
from openai import OpenAI

# Initialize Groq client
Groq_client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"]
)

# Patch instructor to use JSON mode, NOT tool/function calling
#instructor_client = instructor.from_openai(Groq(api_key=groq_api_key))

In [147]:
chat_completion = Groq_client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[{
        "role": "user", 
        "content": 'sup'
    }]
)

chat_completion.choices[0].message.content

"Sup! What's on your mind? Need help with something or just wanna chat? I'm here to listen!"

In [148]:
instructor_client = instructor.from_openai(Groq_client)

In [149]:
from pydantic import BaseModel

class Greeting(BaseModel):
    hello: str

In [150]:
response = instructor_client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[{
        "role": "user",
        "content": "Say hi in JSON"
    }],
    response_model=Greeting,
)
response


Greeting(hello='hi')

### Defining a calendar event

* Simple calendar event extractor to test instructor out. We want the structured output in the below format.

In [151]:
from pydantic import Field
from datetime import date
from typing import List

# Person only has a name, but we can easily extend it with other fields
class Person(BaseModel):
    name: str

class CalendarEvent(BaseModel):
    name: str
    
    # not supported by OpenAI. We want a format like 2025-01-30
    date: date 
    participants: List[Person]

    address_number: str
    street_name: str
    city_name: str

    # Inline regex patterns not supported by OpenAI restrict state code 
    # to be two capital letters (OpenAI does not support pattern fields)
    state_code: str = Field(pattern=r'[A-Z]{2}')

    # Zip code must be five digits
    zip_code: str = Field(pattern=r'\d{5}')

In [152]:
event_description = """
Alice and Bob are going to a science fair on Friday, January 2024.
The science fair is hosted at the gymnasium of Hazeldale Elementary
School at 20080 SW Farmington Road in Beaverton Oregon.
"""

In [153]:
def generate(
    response_model, 
    user_prompt, 
    system_prompt="You are a helpful assistant.",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    max_retries=3,
):
    event = instructor_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_model=response_model,
        max_retries=max_retries
    )

    return event

In [154]:
system_prompt = """
Make a calendar event. Respond in JSON with
the event name, date, list of participants,
and the address.
"""

user_prompt = 'Event description: ' + event_description

event = generate(
    CalendarEvent, 
    user_prompt, 
    system_prompt=system_prompt
)

In [155]:
event

CalendarEvent(name='Science Fair', date=datetime.date(2024, 1, 5), participants=[Person(name='Alice'), Person(name='Bob')], address_number='20080', street_name='SW Farmington Road', city_name='Beaverton', state_code='OR', zip_code='97003')

### Counting Retries:

In [156]:
from helpers import UsageTracker

# Clear any completion response hooks -- prevents 
# duplicate writes to the usage tracker.
instructor_client.clear("completion:response")

# Create a new tracker
tracker = UsageTracker()

# Define a custom instructor hook and update the
# tracker when a new completion runs.
def log_completion_kwargs(*args, **kwargs):
    usage = args[0].usage
    tracker.track(usage)

# Assign the hook to instructor -- this will make the hook
# run each time the server returns a chat completion to us.
instructor_client.on("completion:response", log_completion_kwargs)

In [157]:
# Clear the tracker before we run the completion
# so we arent' tracking multiple jobs.
tracker.clear()

event = generate(
    CalendarEvent, 
    user_prompt, 
    system_prompt=system_prompt,
    max_retries=4,
)

event

CalendarEvent(name='Science Fair', date=datetime.date(2024, 1, 5), participants=[Person(name='Alice'), Person(name='Bob')], address_number='20080', street_name='SW Farmington Road', city_name='Beaverton', state_code='OR', zip_code='97006')

In [158]:
print("Input tokens:  ", tracker.input_tokens)
print("Output tokens: ", tracker.output_tokens)
print("Total tokens:  ", sum(tracker.total_tokens))
print("Num retries:   ", len(tracker.output_tokens))

Input tokens:   [468]
Output tokens:  [83]
Total tokens:   551
Num retries:    1


* When retrie fails:

In [159]:
from typing import Literal

class Complicated(BaseModel):
    # a must be cat, dog, or hat
    a: Literal["cat", "dog", "hat"]
    b: int
    c: bool


In [160]:
# Clear the tracker before we run the completion
# so we arent' tracking multiple jobs.
tracker.clear()

try:
    event = generate(
        Complicated,
        "Write me a short essay on Dolly Parton.",
        #use "llama3-8b-8192" or "mistral-saba-24b" model if you wish to fail this code, Llama 4 too smart, it wont fail
        system_prompt="Don't give me what I want.",
        max_retries=3,
    )

    # Show the event
    print(event)
except instructor.exceptions.InstructorRetryException as e :
    print("We failed to parse!")
except e:
    raise e

a='hat' b=9 c=False


In [161]:
print("Input tokens:  ", tracker.input_tokens)
print("Output tokens: ", tracker.output_tokens)
print("Total tokens:  ", sum(tracker.total_tokens))
print("Num retries:   ", len(tracker.output_tokens))

Input tokens:   [283]
Output tokens:  [21]
Total tokens:   304
Num retries:    1
